In [1]:
import pandas as pd
import numpy as np
from ipywidgets import Dropdown, FloatRangeSlider, DatePicker, SelectMultiple, Button, VBox, HBox, Layout, Label, Textarea
from ipydatagrid import DataGrid, TextRenderer
import datetime as dt

from bqplot import (OrdinalScale, LinearScale, Hist, Figure, Axis, CATEGORY10, Tooltip, Bars)
from bqplot import ColorScale, ColorAxis
from bqplot.interacts import BrushIntervalSelector

from IPython.display import display

In [2]:
import bql
bq = bql.Service()

In [3]:
class visualization:
    def __init__(self, df, num_params):
        
        self.df = df
        self.slided_df = df
        self.colors = ['#ff0000', '#00e6b8','#ff9900','#cce6ff', '#ff00aa', '#8600b3']
        self.num_chart = num_params
        
        self.fig = {}
        self.bin = {}
        self.slider = {}
        self.widgets = {}
        self.plot_init_chart()
        
        
     
        
    def plot_init_chart(self):
        
        for i in range(self.num_chart):
            
            x_ls = LinearScale()
            y_ls = LinearScale(min=0)
            
            plot = Hist(sample=[0], scales={'sample':x_ls, 'count':y_ls}, colors=[self.colors[i]], stroke='black', bins=10,
                        interactions={'click': 'select','hover':'tooltip'},
                        unselected_style={'opacity': .5},
                        tooltip=Tooltip(fields=['count'], labels=['Count'])
                       )
    
            plot.observe(self._bar_click)
    
            xax = Axis(scale=x_ls, grid_lines='none')
            yax = Axis(scale=y_ls, tick_format=',d', grid_lines='none', orientation='vertical')
            
            fig = Figure(marks=[plot], axes=[xax, yax]
                         ,padding_x=0., padding_y=0.
                        )
            
            fig.title = 'Histogram: {}'.format(i)
        
            fig.layout.width, fig.layout.height = '400px', '220px'
            fig.fig_margin = {'bottom':40, 'left':60, 'right':20, 'top':40}
            
            self.fig[i] = fig
            
            bin_picker = Dropdown(options=[10,20,50,100],description='bar number',style={'description_width':'initial'},value=10
                             ,layout=Layout(width='150px'))
            bin_picker.observe(self._bin_change,'value')
            self.bin[i] = bin_picker
            
            range_slider = FloatRangeSlider(
                                            value=[0, 0],
                                            min=0,
                                            max=0,
                                            step=0.1,
                                            description='Range:',
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
#                                             readout=True,
#                                             readout_format='.2f',
                                            layout=Layout(width='400px')
                                        )
            range_slider.observe(self._slider_filter_update,'value')
            self.slider[i] = range_slider
            
            
            self.widgets[i] = VBox([bin_picker, range_slider, fig])
            
            self.datagrid = DataGrid(self.df, base_column_size=80, layout={'height':'300px'},
                                    column_widths={'ID':120, 'Company Name':180, 'GICS Sector':120},
                                    renderers={
                                              'DATE': TextRenderer(format='%Y/%m/%d', format_type='time'),
                                              'AS_OF_DATE': TextRenderer(format='%Y/%m/%d', format_type='time'),
                                              'Market Cap': TextRenderer(format=',.2f'),
                                              #IS
                                              'Revenue': TextRenderer(format=',.2f'),
                                              'Net Income': TextRenderer(format=',.2f'),
                                              #IS ratio
                                              'Gross Margin':TextRenderer(format='.2f'),
                                              'Operating Margin':TextRenderer(format='.2f'),
                                              'NI Margin':TextRenderer(format='.2f'),
                                              #BS
                                              'Cash & Equiv':TextRenderer(format=',.2f'),
                                              'CA':TextRenderer(format=',.2f'),
                                              #BS ratio
                                             
                                              #exchange
                                              'Price':TextRenderer(format='.2f'),
                                              'Volume':TextRenderer(format='.2d'),
                                              #other ratios
                                              'PE Ratio': TextRenderer(format='.2f'),
                                              'ROE':TextRenderer(format='.2f'),
                                              'ROA':TextRenderer(format='.2f'),
                                              'EPS':TextRenderer(format='.2f'),
                                              'Dvd Payout Ratio':TextRenderer(format='.2f')
                                              }
                                    )
            
        
        
        self.widgets['vs'] = VBox([
                                    HBox([self.widgets[0], self.widgets[1], self.widgets[2]]),
                                    HBox([self.widgets[3], self.widgets[4], self.widgets[5]]),
                                    self.datagrid
        ])
        
            
            
    def _bin_change(self, *arg):
        
        for i in range(self.num_chart):
            if self.bin[i].value!=self.fig[i].marks[0].bins:
                self.fig[i].marks[0].bins = self.bin[i].value
                self.slider[i].step= (self.slider[i].max-self.slider[i].min)/(self.bin[i].value+1)
            
            
            
    def _bar_click(self, evt=None):
        
        dg_index = list(range(len(self.slided_df.index))) 

        for i in range(self.num_chart):
            if (np.nansum(self.fig[i].marks[0].sample)>0) and (self.fig[i].marks[0].selected is not None): #not empty
                dg_index = list(set(dg_index).intersection(list(self.fig[i].marks[0].selected)))

        self.datagrid.transform([{'type': 'filter',
                             'columnIndex': 0,
                             'operator': 'in',
                             'value': list(self.slided_df.iloc[dg_index].index)}])
    
    
    def _chart_update(self, df):   
        
        global select_params
        
        for i in range(len(select_params.value)):
            param = list(select_params.value)[i]
            self.fig[i].title = 'Parameters: {}'.format(param)
            self.fig[i].marks[0].sample = df[param]
            self.fig[i].marks[0].selected = None
                       
        self.datagrid.data = df
    
    
    def _slider_chart_update(self, df):
        
        global select_params
        
        self.df = df
        self.slided_df = df
        
        
        for i in range(len(select_params.value)):
            param = list(select_params.value)[i]
            self.slider[i].unobserve(self._slider_filter_update,'value')
            if np.nanmax(df[param])>=self.slider[i].min:
                self.slider[i].max = np.nanmax(df[param])+1
                self.slider[i].min = np.nanmin(df[param])-1
            elif np.nanmin(df[param])<=self.slider[i].max:
                self.slider[i].min = np.nanmin(df[param])-1
                self.slider[i].max = np.nanmax(df[param])+1
            self.slider[i].value = [self.slider[i].min,self.slider[i].max]
            self.slider[i].step = (self.slider[i].max-self.slider[i].min)/(self.bin[i].value+1)
            self.slider[i].observe(self._slider_filter_update,'value')
            
        self._chart_update(df)
        
        
    def _slider_filter_update(self, evt=None):
        
        slider_idx = self.df.index!=False
        
        for i in range(len(select_params.value)):
            param = list(select_params.value)[i]
            slider_idx = slider_idx & ((self.slider[i].value[0]<=self.df[param]) & (self.df[param]<=self.slider[i].value[1]) | self.df[param].isna())
        
        self.slided_df = self.df[slider_idx]
        self._chart_update(self.slided_df)
        
        
    

In [4]:
def get_data(evt):
    
    global vs
    global select_params
    
    if len(select_params.value)>vs.num_chart:
        pass
    else:
        filtered_tickers, bql_params = filter_ticker_param()
        df = query_data(filtered_tickers, bql_params)
        vs._slider_chart_update(df)
        
        
def click_add(evt):
    
    global vs
    global text_area
    global df_add
    
    ticker_list = [i.strip()+' Equity' for i in text_area.value.replace(',','\n').split('\n')]
    to_add_list = set(ticker_list).difference(set(vs.df.index))
    bql_tickers = bq.univ.list(list(to_add_list))
    
    params = vs.df.columns[1:]
    bql_params = {i:full_params[i] for i in params}
    
    df_add = query_data(bql_tickers,bql_params)
    
    vs.df = pd.concat([vs.df, df_add], axis=0)
    vs._slider_chart_update(vs.df)
    
    
def click_del(evt):
    
    global vs
    global text_area
    
    ticker_list = [i.strip()+' Equity' for i in text_area.value.replace(',','\n').split('\n')]
    to_drop_list = list(set(ticker_list) & set(vs.df.index))
    vs._slider_chart_update(vs.df.drop(to_drop_list, axis=0))
    
    


def filter_ticker_param():
    #process ticker filter and params filter
    global market_picker, select_sector
    global full_params
    
    bql_item = bq.data.name(MODE='CACHED')
    
    bql_universe = bq.univ.filter(bq.univ.equitiesuniv(['active', 'primary']), 
                                  bq.func.and_(
                                      bq.data.exch_code()==market_picker.value, 
                                      bq.func.in_(bq.data.gics_sector_name(), list(select_sector.value))
                                  )
                                 )
    
    bql_request = bql.Request(bql_universe, bql_item)
    bql_response = bq.execute(bql_request)
    
#     filtered_tickers = list(bql_response[0].df().index)
    filtered_tickers = bq.univ.list(bql_response[0].df().index)
    
    params = ['Company Name','GICS Sector']
    params.extend(list(select_params.value))
    bql_params = {i:full_params[i] for i in params}
    
    
    return filtered_tickers, bql_params


def query_data(tickers, params):

    request = bql.Request(tickers, params)
    response = bq.execute(request)

    response_list = [response[i].df() for i in range(len(response))]
    df_res = pd.concat(response_list, axis=1)

    mark = []
    if 'DATE' in df_res.columns:
        mark.append('DATE')
    elif 'AS_OF_DATE'  in df_res.columns:
        mark.append('AS_OF_DATE')
        
    mark.extend(list(params.keys()))
    df_ret = df_res[mark]
    df_ret = df_ret.iloc[:,~df_ret.columns.duplicated()]

    return df_ret

In [5]:
market_options = ['CH', 'HK']
select_date = dt.datetime.now().strftime("%Y-%m-%d")
gics_list = ['Communication Services','Consumer Discretionary','Consumer Staples','Energy','Financials',
             'Health Care','Industrials','Information Technology','Materials','Real Estate','Utilities']

widget_width = '180px'

market_picker = Dropdown(options=market_options,style={'description_width':'initial'},value=market_options[0]
                             ,layout=Layout(width=widget_width))
label_market_picker = Label(value='Market', font_weight='bolder')

date_picker = DatePicker(disabled=False,value=dt.datetime.now()
                         ,layout=Layout(width=widget_width))
label_date_picker = Label(value='Pick a Date', font_weight='bolder')

date = date_picker.value.strftime("%Y-%m-%d")

full_params = {
                'Company Name': bq.data.name(),
                'GICS Sector': bq.data.gics_sector_name(),
                'Market Cap': bq.data.cur_mkt_cap(DATES=date),
                #IS
                'Revenue': bq.data.sales_rev_turn(DATES=date),
                'Net Income': bq.data.earn_for_common(DATES=date),
                #IS ratio
                'Gross Margin': bq.data.gross_margin(DATES=date),
                'Operating Margin': bq.data.oper_margin(DATES=date),
                'NI Margin': bq.data.prof_margin(DATES=date),
                #BS
                'Cash & Equiv': bq.data.c_and_ce_and_sti_detailed(DATES=date),
                'CA': bq.data.bs_cur_asset_report(DATES=date),
                #BS ratio
    
                #exchange
                'Price': bq.data.px_last(DATES=date),
                'Volume': bq.data.px_volume(DATES=date),
                #other ratios
                'PE Ratio': bq.data.pe_ratio(DATES=date),
                'ROE': bq.data.return_com_eqy(DATES=date),
                'ROA': bq.data.return_on_asset(DATES=date),
                'EPS': bq.data.is_eps(DATES=date),
                'Dvd Payout Ratio': bq.data.dvd_payout_ratio(DATES=date)                
}

params = list(full_params.keys())[2:8]


select_params = SelectMultiple(options=list(full_params.keys())[2:],value=params,disabled=False
                                                ,layout=Layout(width=widget_width, height='200px')
                           )
label_select_params = Label(value='Parameters', font_weight='bolder')

select_sector = SelectMultiple(options=gics_list, value=gics_list,disabled=False
                                                ,layout=Layout(width=widget_width)
                              )
label_select_sector = Label(value='Sector', font_weight='bolder')

button_update = Button(button_style ='success',description='Get Data'
                      ,layout=Layout(width=widget_width))
text_area = Textarea(
                    value='',
                    placeholder='Update ticker here',
                #     description='String',
                    disabled=False,
                    layout=Layout(width=widget_width,height='100px'),
                    resize=None
                )
label_text_area = Label(value='Ticker updater', font_weight='bolder')
button_add = Button(button_style ='success',description='Add'
                      ,layout=Layout(width='90px'))
button_del = Button(button_style ='success',description='Del'
                      ,layout=Layout(width='90px'))    

label_notice = Label(value='', font_weight='bolder')


df = pd.DataFrame([],index=['NA'],columns=['Company Name','GICS Sector'])   #place holder
vs = visualization(df, len(select_params.value))

button_update.on_click(get_data)
button_add.on_click(click_add)
button_del.on_click(click_del)


interface = HBox([
                    VBox([label_select_params,select_params,label_market_picker,market_picker,
                          label_date_picker,date_picker,label_select_sector,select_sector,button_update
                          ,label_text_area,text_area
                          ,HBox([button_add,button_del])
                          ,label_notice])
                    ,vs.widgets['vs']
])
                
display(interface)
